In [1]:
import sys
import numpy as np
from versioned_hdf5.staged_changes import StagedChangesArray, _fmt_fancy_index

print(sys.modules["versioned_hdf5.hyperspace"])
print(sys.modules["versioned_hdf5.staged_changes"])
print(sys.modules["versioned_hdf5.subchunk_map"])

<module 'versioned_hdf5.hyperspace' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/hyperspace.cpython-312-x86_64-linux-gnu.so'>
<module 'versioned_hdf5.staged_changes' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/staged_changes.cpython-312-x86_64-linux-gnu.so'>
<module 'versioned_hdf5.subchunk_map' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/subchunk_map.cpython-312-x86_64-linux-gnu.so'>


# StagedChangesArray demo

In [2]:
base = np.arange(64).reshape(8, 8)
def base_getitem(idx):
    print(f"Reading from base: [{_fmt_fancy_index(idx)}]")
    return base[idx]

s = StagedChangesArray(
    base_getitem,
    shape=base.shape,
    dtype=base.dtype,
    chunk_size=(2, 2),
)
s

StagedChangesArray<shape=(8, 8), chunk_size=(2, 2), dtype=int64, fill_value=0, 0 modified chunks>
chunk_states:
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]

### \_\_setitem\_\_()

In [3]:
s._setitem_plan((slice(2, 5), slice(3, 6)))

SetItemPlan<shape=(3, 3), 2 loads from base into 3 chunks, 0 appends of empty chunks, 0 appends of full chunks, 1 appends from __setitem__ value, 0 replaces with empty chunks, 0 replaces from __setitem__ value, 3 updates>
  chunk_values.append(base[2:4, 2:4])  # [1]
  v0 = base[4:6, 2:6]
  chunk_values.append(v0[:, 0:2])  # [2]
  chunk_values.append(v0[:, 2:4])  # [3]
  chunk_values.append(values[0:2, 1:3])  # [4]
  chunk_values[1][0:2, 1:2] = values[0:2, 0:1]
  chunk_values[2][0:1, 1:2] = values[2:3, 0:1]
  chunk_values[3][0:1, 0:2] = values[2:3, 1:3]
chunk_states:
[[0 0 0 0]
 [0 1 4 0]
 [0 2 3 0]
 [0 0 0 0]]

In [4]:
s[2:5, 3:6] = np.arange(9).reshape(3, 3) + 1000
s

Reading from base: [2:4, 2:4]
Reading from base: [4:6, 2:6]


StagedChangesArray<shape=(8, 8), chunk_size=(2, 2), dtype=int64, fill_value=0, 4 modified chunks>
chunk_states:
[[0 0 0 0]
 [0 1 4 0]
 [0 2 3 0]
 [0 0 0 0]]

### changes()

In [5]:
s._changes_plan()

ChangesPlan<0 deleted chunks, 4 chunks in memory, 0 loads from base into 0 chunks>
  base[2:4, 2:4] = chunk_values[1]
  base[2:4, 4:6] = chunk_values[4]
  base[4:6, 2:4] = chunk_values[2]
  base[4:6, 4:6] = chunk_values[3]

In [6]:
list(s.changes())

[((slice(2, 4, 1), slice(2, 4, 1)),
  array([[  18, 1000],
         [  26, 1003]])),
 ((slice(2, 4, 1), slice(4, 6, 1)),
  array([[1001, 1002],
         [1004, 1005]])),
 ((slice(4, 6, 1), slice(2, 4, 1)),
  array([[  34, 1006],
         [  42,   43]])),
 ((slice(4, 6, 1), slice(4, 6, 1)),
  array([[1007, 1008],
         [  44,   45]]))]

### \_\_getitem\_\_()

In [7]:
s._getitem_plan(())

GetItemPlan<shape=(8, 8), 4 modified chunks, 0 full chunks, 6 direct reads>
  res[2:4, 2:4] = chunk_values[1][0:2, 0:2]
  res[2:4, 4:6] = chunk_values[4][0:2, 0:2]
  res[4:6, 2:4] = chunk_values[2][0:2, 0:2]
  res[4:6, 4:6] = chunk_values[3][0:2, 0:2]
  res[0:2, 0:8] = base[0:2, 0:8]
  res[2:4, 0:2] = base[2:4, 0:2]
  res[2:4, 6:8] = base[2:4, 6:8]
  res[4:6, 0:2] = base[4:6, 0:2]
  res[4:6, 6:8] = base[4:6, 6:8]
  res[6:8, 0:8] = base[6:8, 0:8]

In [8]:
s[:]

Reading from base: [0:2, 0:8]
Reading from base: [2:4, 0:2]
Reading from base: [2:4, 6:8]
Reading from base: [4:6, 0:2]
Reading from base: [4:6, 6:8]
Reading from base: [6:8, 0:8]


array([[   0,    1,    2,    3,    4,    5,    6,    7],
       [   8,    9,   10,   11,   12,   13,   14,   15],
       [  16,   17,   18, 1000, 1001, 1002,   22,   23],
       [  24,   25,   26, 1003, 1004, 1005,   30,   31],
       [  32,   33,   34, 1006, 1007, 1008,   38,   39],
       [  40,   41,   42,   43,   44,   45,   46,   47],
       [  48,   49,   50,   51,   52,   53,   54,   55],
       [  56,   57,   58,   59,   60,   61,   62,   63]])

In [9]:
# __getitem__ loads data transparently from base to output
# (no cache on read)
s

StagedChangesArray<shape=(8, 8), chunk_size=(2, 2), dtype=int64, fill_value=0, 4 modified chunks>
chunk_states:
[[0 0 0 0]
 [0 1 4 0]
 [0 2 3 0]
 [0 0 0 0]]

### resize()
When enlarging, new chunks full of fillvalue all point to `chunk_values[0]`.

It may also be necessary to load partially resized chunks from base.

In [10]:
s._resize_plan((5, 9))

ResizePlan<2 loads from base into 2 chunks, 4 updates, 0 deletes>
  chunk_values.append(base[4:6, 0:2])  # [5]
  chunk_values.append(base[4:6, 6:8])  # [6]
  resize chunk_values[5] to size=1 along axis=0
  resize chunk_values[2] to size=1 along axis=0
  resize chunk_values[3] to size=1 along axis=0
  resize chunk_values[6] to size=1 along axis=0
chunk_states:
[[ 0  0  0  0 -1]
 [ 0  1  4  0 -1]
 [ 5  2  3  6 -1]]

In [11]:
s.resize((5, 9))

Reading from base: [4:6, 0:2]
Reading from base: [4:6, 6:8]


In [12]:
s

StagedChangesArray<shape=(5, 9), chunk_size=(2, 2), dtype=int64, fill_value=0, 6 modified chunks>
chunk_states:
[[ 0  0  0  0 -1]
 [ 0  1  4  0 -1]
 [ 5  2  3  6 -1]]

In [13]:
s[:]

Reading from base: [0:2, 0:8]
Reading from base: [2:4, 0:2]
Reading from base: [2:4, 6:8]


array([[   0,    1,    2,    3,    4,    5,    6,    7,    0],
       [   8,    9,   10,   11,   12,   13,   14,   15,    0],
       [  16,   17,   18, 1000, 1001, 1002,   22,   23,    0],
       [  24,   25,   26, 1003, 1004, 1005,   30,   31,    0],
       [  32,   33,   34, 1006, 1007, 1008,   38,   39,    0]])

In [14]:
# __setitem__() encroaching on full chunks
# (wholly on chunk row 0, partially on chunk row 1)
s._setitem_plan((slice(0, 3), -1))

SetItemPlan<shape=(3,), 0 loads from base into 0 chunks, 1 appends of empty chunks, 1 appends of full chunks, 0 appends from __setitem__ value, 0 replaces with empty chunks, 0 replaces from __setitem__ value, 2 updates>
  chunk_values.append(np.empty((2, 1))  # [7]
  chunk_values.append(np.full((2, 1), fill_value)  # [8]
  chunk_values[7][0:2, 0] = values[0:2]
  chunk_values[8][0:1, 0] = values[2:3]
chunk_states:
[[ 0  0  0  0  7]
 [ 0  1  4  0  8]
 [ 5  2  3  6 -1]]

In [15]:
s[:3, -1] = 2000

In [16]:
s

StagedChangesArray<shape=(5, 9), chunk_size=(2, 2), dtype=int64, fill_value=0, 8 modified chunks>
chunk_states:
[[ 0  0  0  0  7]
 [ 0  1  4  0  8]
 [ 5  2  3  6 -1]]

In [17]:
s[:]

Reading from base: [0:2, 0:8]
Reading from base: [2:4, 0:2]
Reading from base: [2:4, 6:8]


array([[   0,    1,    2,    3,    4,    5,    6,    7, 2000],
       [   8,    9,   10,   11,   12,   13,   14,   15, 2000],
       [  16,   17,   18, 1000, 1001, 1002,   22,   23, 2000],
       [  24,   25,   26, 1003, 1004, 1005,   30,   31,    0],
       [  32,   33,   34, 1006, 1007, 1008,   38,   39,    0]])

In [18]:
s._changes_plan()

ChangesPlan<8 deleted chunks, 8 chunks in memory, 0 loads from base into 0 chunks>
  del base[4:6, 0:2]
  del base[4:6, 2:4]
  del base[4:6, 4:6]
  del base[4:6, 6:8]
  del base[6:8, 0:2]
  del base[6:8, 2:4]
  del base[6:8, 4:6]
  del base[6:8, 6:8]
  base[0:2, 8:9] = chunk_values[7]
  base[2:4, 2:4] = chunk_values[1]
  base[2:4, 4:6] = chunk_values[4]
  base[2:4, 8:9] = chunk_values[8]
  base[4:5, 0:2] = chunk_values[5]
  base[4:5, 2:4] = chunk_values[2]
  base[4:5, 4:6] = chunk_values[3]
  base[4:5, 6:8] = chunk_values[6]

### astype()
StagedChangesArray.astype() is lazy: it won't read from disk until you call changes().

In [19]:
s2 = s.astype(float)

In [20]:
s2

StagedChangesArray<shape=(5, 9), chunk_size=(2, 2), dtype=float64, fill_value=0.0, 8 modified chunks>
chunk_states:
[[ 0  0  0  0  7]
 [ 0  1  4  0  8]
 [ 5  2  3  6 -1]]

In [21]:
s2._changes_plan()

ChangesPlan<8 deleted chunks, 8 chunks in memory, 3 loads from base into 6 chunks>
  del base[4:6, 0:2]
  del base[4:6, 2:4]
  del base[4:6, 4:6]
  del base[4:6, 6:8]
  del base[6:8, 0:2]
  del base[6:8, 2:4]
  del base[6:8, 4:6]
  del base[6:8, 6:8]
  base[0:2, 8:9] = chunk_values[7]
  base[2:4, 2:4] = chunk_values[1]
  base[2:4, 4:6] = chunk_values[4]
  base[2:4, 8:9] = chunk_values[8]
  base[4:5, 0:2] = chunk_values[5]
  base[4:5, 2:4] = chunk_values[2]
  base[4:5, 4:6] = chunk_values[3]
  base[4:5, 6:8] = chunk_values[6]
  v0 = base[0:2, 0:8]
  out[0:2, 0:2] = v0[:, 0:2]
  out[0:2, 2:4] = v0[:, 2:4]
  out[0:2, 4:6] = v0[:, 4:6]
  out[0:2, 6:8] = v0[:, 6:8]
  out[2:4, 0:2] = base[2:4, 0:2]
  out[2:4, 6:8] = base[2:4, 6:8]

### Sparse arrays

In [22]:
s_sparse = StagedChangesArray.full(
    shape=(5, 8),
    chunk_size=(2, 2),
)
s_sparse[3, 2] = 1337
s_sparse

StagedChangesArray<shape=(5, 8), chunk_size=(2, 2), dtype=float64, fill_value=0.0, 1 modified chunks>
chunk_states:
[[-1 -1 -1 -1]
 [-1  1 -1 -1]
 [-1 -1 -1 -1]]

In [23]:
s_sparse[:]

array([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0., 1337.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]])

In [24]:
s_sparse._changes_plan()

ChangesPlan<0 deleted chunks, 1 chunks in memory, 0 loads from base into 0 chunks>
  base[2:4, 2:4] = chunk_values[1]

In [25]:
list(s_sparse.changes())

[((slice(2, 4, 1), slice(2, 4, 1)),
  array([[   0.,    0.],
         [1337.,    0.]]))]

### Copy-on-Write (CoW)

In [26]:
s2 = s.copy()
s2.resize((5, 5))
s2[3, 4] = 999
s[0, 3] = 0
s2[:]

Reading from base: [0:2, 4:6]
Reading from base: [0:2, 2:4]
Reading from base: [0:2, 0:4]
Reading from base: [2:4, 0:2]


array([[   0,    1,    2,    3,    4],
       [   8,    9,   10,   11,   12],
       [  16,   17,   18, 1000, 1001],
       [  24,   25,   26, 1003,  999],
       [  32,   33,   34, 1006, 1007]])

In [27]:
s[:]

Reading from base: [0:2, 0:2]
Reading from base: [0:2, 4:8]
Reading from base: [2:4, 0:2]
Reading from base: [2:4, 6:8]


array([[   0,    1,    2,    0,    4,    5,    6,    7, 2000],
       [   8,    9,   10,   11,   12,   13,   14,   15, 2000],
       [  16,   17,   18, 1000, 1001, 1002,   22,   23, 2000],
       [  24,   25,   26, 1003, 1004, 1005,   30,   31,    0],
       [  32,   33,   34, 1006, 1007, 1008,   38,   39,    0]])

In [28]:
[a is b for a, b in zip(s.chunk_values, s2.chunk_values)]

[True, True, True, False, False, True, False, False, False, False]

In [29]:
# Chunk (0, 0) is not in memory so it is loaded and refilled lazily as needed
# Chunk (0, 1) is in memory and partially full, so it is copied and refilled eagerly
# Bottom-right chunk is full of fill_value, so it is lazily refilled
s3 = s.refill(777)
s3[:]

Reading from base: [0:2, 0:2]
Reading from base: [0:2, 4:8]
Reading from base: [2:4, 0:2]
Reading from base: [2:4, 6:8]


array([[ 777,    1,    2,  777,    4,    5,    6,    7, 2000],
       [   8,    9,   10,   11,   12,   13,   14,   15, 2000],
       [  16,   17,   18, 1000, 1001, 1002,   22,   23, 2000],
       [  24,   25,   26, 1003, 1004, 1005,   30,   31,  777],
       [  32,   33,   34, 1006, 1007, 1008,   38,   39,  777]])

In [30]:
[a is b for a, b in zip(s.chunk_values, s3.chunk_values)]

[True, True, True, True, True, True, True, True, False, False]

### The fill_hyperspace function

In [31]:
from versioned_hdf5.hyperspace import fill_hyperspace
from versioned_hdf5.subchunk_map import EverythingMapper
from versioned_hdf5.staged_changes import _modified_chunks_in_selection

space = np.zeros((8, 10), dtype=int)
space[2, 4] = 1
space[2, 5] = 3
space[5, 7] = 2
space

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [32]:
modified_chunks, _ = _modified_chunks_in_selection(
    space,
    [EverythingMapper(10, s * 10) for s in space.shape],
)
modified_chunks

array([[2, 4, 1],
       [2, 5, 3],
       [5, 7, 2]])

In [33]:
ndim = space.ndim
obstacles = modified_chunks[:, :ndim]
hyperrectangles = fill_hyperspace(obstacles=obstacles, shape=(8, 10))
np.asarray(hyperrectangles)

array([[ 0,  0,  2, 10],
       [ 2,  0,  3,  4],
       [ 2,  6,  3, 10],
       [ 3,  0,  5, 10],
       [ 5,  0,  6,  7],
       [ 5,  8,  6, 10],
       [ 6,  0,  8, 10]])

In [34]:
i = 10
for hyperrect in hyperrectangles:
    idx = tuple(
        slice(a, b)
        for a, b in zip(hyperrect[:ndim], hyperrect[ndim:])
    )
    assert (space[idx] == 0).all()
    space[idx] = i
    i += 1

space

array([[10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [11, 11, 11, 11,  1,  3, 12, 12, 12, 12],
       [13, 13, 13, 13, 13, 13, 13, 13, 13, 13],
       [13, 13, 13, 13, 13, 13, 13, 13, 13, 13],
       [14, 14, 14, 14, 14, 14, 14,  2, 15, 15],
       [16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
       [16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])

In [35]:
shape = (1000, 1000)  # 1 million chunks
n_obstacles = 100_000  # 10%
space = np.zeros(shape, dtype=np.intp)
np.random.seed(0)
obstacles = np.random.choice(np.arange(space.size), n_obstacles)
space.flat[obstacles] = np.arange(1, space.size + 1)
mappers = [EverythingMapper(10, s * 10) for s in space.shape]

%time modified_chunks, _ = _modified_chunks_in_selection(space, mappers)
modified_chunks

CPU times: user 4.42 ms, sys: 21 μs, total: 4.44 ms
Wall time: 4.46 ms


array([[    0,     4, 93094],
       [    0,    27, 43556],
       [    0,    76, 82239],
       ...,
       [  999,   916, 18975],
       [  999,   994, 87212],
       [  999,   997, 62516]])

In [36]:
obstacles = modified_chunks[:, :ndim]
%time hyperrectangles = fill_hyperspace(obstacles=obstacles, shape=space.shape)
print(len(hyperrectangles))
np.asarray(hyperrectangles)

CPU times: user 22.9 ms, sys: 2.97 ms, total: 25.9 ms
Wall time: 25.6 ms
86870


array([[   0,    0,    1,    4],
       [   0,    5,    1,   27],
       [   0,   28,    1,   76],
       ...,
       [ 999,  917, 1000,  994],
       [ 999,  995, 1000,  997],
       [ 999,  998, 1000, 1000]])